# Polarization Manifestation Classifier (Subtask 3)

**Multi-Label Classification for Arabic Text**

This notebook classifies Arabic text snippets across 6 manifestation types:
- **Stereotype**: Generalizations about groups
- **Vilification**: Abusive or defamatory language
- **Dehumanization**: Denying human qualities
- **Extreme Language**: Inflammatory or aggressive language
- **Lack of Empathy**: Dismissive of others' experiences
- **Invalidation**: Denying legitimacy of perspectives

**Approach**: ACE-GPT (AraGPT2) with few-shot learning and culturally-aware prompting

---

## Part 1: Setup & Configuration

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import torch
import json
import re
from typing import List, Dict, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import f1_score, hamming_loss, precision_recall_fscore_support
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Configuration
class Config:
    model_name = "aubmindlab/aragpt2-medium"
    max_length = 1024
    temperature = 0.3  # Lower for consistency
    top_p = 0.9
    max_new_tokens = 100
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 42
    eval_samples = 30
    data_path = "../arb.csv"

config = Config()
np.random.seed(config.seed)
torch.manual_seed(config.seed)

print("=" * 70)
print("CONFIGURATION")
print("=" * 70)
print(f"Model: {config.model_name}")
print(f"Device: {config.device}")
print(f"Temperature: {config.temperature}")
print(f"Max Length: {config.max_length}")
print(f"Eval Samples: {config.eval_samples}")
print(f"Data Path: {config.data_path}")
print("=" * 70)

In [ ]:
# Load dataset
df = pd.read_csv(config.data_path)

# Define manifestation types
MANIFESTATION_TYPES = [
    'stereotype', 
    'vilification', 
    'dehumanization', 
    'extreme_language', 
    'lack_of_empathy', 
    'invalidation'
]

print("=" * 70)
print("DATASET OVERVIEW")
print("=" * 70)
print(f"Total samples: {len(df):,}")
print(f"\nManifestations: {MANIFESTATION_TYPES}")
print(f"\nLabel distribution:")
for label in MANIFESTATION_TYPES:
    positive_count = df[label].sum()
    print(f"  {label:20s}: {positive_count:5,} ({positive_count/len(df)*100:.1f}%)")
print("=" * 70)

# Show sample
print("\nSample data:")
df.head(3)

## Part 2: Manifestation Context Mapper

Maps each manifestation type to Arabic names and contextual descriptions for prompting.

In [ ]:
class ManifestationContextMapper:
    """Maps manifestation types to Arabic names and contextual descriptions."""
    
    def __init__(self):
        self.manifestation_contexts = {
            'stereotype': {
                'ar_name': 'القوالب النمطية',
                'context': 'القوالب النمطية هي تعميمات مبسطة عن مجموعة من الناس',
                'examples': ['كل X كسالى', 'دائماً Y يفعلون Z', 'المعروف عن هؤلاء']
            },
            'vilification': {
                'ar_name': 'التشويه والإهانة',
                'context': 'التشويه يشمل لغة مسيئة أو تشهيرية تهاجم شخص أو مجموعة',
                'examples': ['شتائم', 'إهانات', 'تحقير']
            },
            'dehumanization': {
                'ar_name': 'التجريد من الإنسانية',
                'context': 'التجريد من الإنسانية ينكر الصفات الإنسانية ويشبه الناس بالحيوانات',
                'examples': ['كلاب', 'حيوانات', 'وحوش', 'قطيع']
            },
            'extreme_language': {
                'ar_name': 'اللغة المتطرفة',
                'context': 'اللغة المتطرفة تشمل كلمات عدوانية أو تحريضية قوية',
                'examples': ['يجب القضاء على', 'لا يستحقون الحياة', 'يستحقون الموت']
            },
            'lack_of_empathy': {
                'ar_name': 'عدم التعاطف',
                'context': 'عدم التعاطف يتجاهل أو يستهزئ بمعاناة الآخرين',
                'examples': ['يستحقون ما حدث', 'لا أهتم', 'مشكلتهم']
            },
            'invalidation': {
                'ar_name': 'الإلغاء والإنكار',
                'context': 'الإلغاء ينكر شرعية وجهات نظر أو تجارب الآخرين',
                'examples': ['هذا غير صحيح', 'لا يحق لهم', 'رأيهم باطل']
            }
        }
    
    def get_ar_name(self, manifestation: str) -> str:
        """Get Arabic name for manifestation."""
        return self.manifestation_contexts.get(manifestation, {}).get('ar_name', manifestation)
    
    def get_context(self, manifestation: str) -> str:
        """Get context description for manifestation."""
        return self.manifestation_contexts.get(manifestation, {}).get('context', '')
    
    def get_examples(self, manifestation: str) -> List[str]:
        """Get example indicators for manifestation."""
        return self.manifestation_contexts.get(manifestation, {}).get('examples', [])

# Initialize mapper
manifestation_mapper = ManifestationContextMapper()

print("✓ Manifestation Context Mapper initialized")
print("\nManifestation Arabic Names:")
for manif in MANIFESTATION_TYPES:
    print(f"  {manif:20s} → {manifestation_mapper.get_ar_name(manif)}")

## Part 3: Few-Shot Example Bank

Builds a bank of positive and negative examples for each manifestation type from the training data.

In [ ]:
class FewShotExampleBank:
    """Builds and manages few-shot examples for each manifestation type."""
    
    def __init__(self, df: pd.DataFrame, labels: List[str]):
        self.df = df
        self.labels = labels
        self.example_bank = self._build_example_bank()
    
    def _build_example_bank(self) -> Dict:
        """Build example bank with positive and negative samples."""
        bank = {label: {'positive': [], 'negative': []} for label in self.labels}
        
        for _, row in self.df.iterrows():
            text = row['text']
            if pd.isna(text) or len(text) > 200:  # Skip too long texts
                continue
            
            for label in self.labels:
                label_val = row[label]
                if pd.isna(label_val):
                    continue
                
                # Collect positive examples
                if label_val == 1 and len(bank[label]['positive']) < 100:
                    bank[label]['positive'].append({'text': text, 'label': 1})
                
                # Collect negative examples (where this label is 0 and no other label is 1)
                elif label_val == 0 and len(bank[label]['negative']) < 100:
                    other_labels_sum = sum([row[l] for l in self.labels if pd.notna(row[l]) and l != label])
                    if other_labels_sum == 0:
                        bank[label]['negative'].append({'text': text, 'label': 0})
        
        return bank
    
    def get_few_shot_examples(self, manifestation: str, n: int = 2) -> List[Dict]:
        """Get balanced few-shot examples for a manifestation."""
        positive_examples = self.example_bank[manifestation]['positive']
        negative_examples = self.example_bank[manifestation]['negative']
        
        # Sample positive examples
        pos_sample = np.random.choice(
            len(positive_examples), 
            size=min(n, len(positive_examples)), 
            replace=False
        ) if len(positive_examples) > 0 else []
        
        # Sample negative examples
        neg_sample = np.random.choice(
            len(negative_examples), 
            size=min(n, len(negative_examples)), 
            replace=False
        ) if len(negative_examples) > 0 else []
        
        # Combine examples
        examples = []
        for idx in pos_sample:
            examples.append(positive_examples[idx])
        for idx in neg_sample:
            examples.append(negative_examples[idx])
        
        np.random.shuffle(examples)
        return examples

# Build example bank
few_shot_bank = FewShotExampleBank(df, MANIFESTATION_TYPES)

print("✓ Few-Shot Example Bank built")
print("\nExample bank statistics:")
for manif in MANIFESTATION_TYPES:
    pos_count = len(few_shot_bank.example_bank[manif]['positive'])
    neg_count = len(few_shot_bank.example_bank[manif]['negative'])
    print(f"  {manif:20s}: {pos_count} positive, {neg_count} negative")

## Part 4: Improved Prompter

Creates culturally-aware prompts for each manifestation type with few-shot examples.

In [ ]:
class ImprovedPrompter:
    """Creates culturally-aware prompts for manifestation detection."""
    
    def __init__(self, manifestation_mapper: ManifestationContextMapper):
        self.manifestation_mapper = manifestation_mapper
    
    def get_manifestation_hints(self, manifestation: str) -> str:
        """Get specific guidance for each manifestation to reduce false positives."""
        hints = {
            'stereotype': 'ملاحظة: يجب أن يحتوي على تعميم واضح عن مجموعة من الناس.',
            'vilification': 'ملاحظة: يجب أن يحتوي على شتائم أو إهانات مباشرة.',
            'dehumanization': 'ملاحظة: يجب أن يشبه الناس بالحيوانات أو ينكر إنسانيتهم.',
            'extreme_language': 'ملاحظة: يجب أن يحتوي على لغة عدوانية أو تحريضية قوية.',
            'lack_of_empathy': 'ملاحظة: يجب أن يتجاهل أو يستهزئ بمعاناة الآخرين.',
            'invalidation': 'ملاحظة: يجب أن ينكر شرعية وجهات نظر أو تجارب الآخرين.'
        }
        return hints.get(manifestation, '')
    
    def create_completion_prompt(self, text: str, manifestation: str, few_shot_examples: str = "") -> str:
        """Create completion-style prompt for the model."""
        ar_name = self.manifestation_mapper.get_ar_name(manifestation)
        hint = self.get_manifestation_hints(manifestation)
        
        prompt = f"""{few_shot_examples}{hint}

النص: "{text}"
السؤال: هل يحتوي على {ar_name}؟
الإجابة:"""
        
        return prompt
    
    def format_few_shot_with_reasoning(self, examples: List[Dict], manifestation: str) -> str:
        """Format few-shot examples with reasoning to teach the model."""
        if not examples:
            return ""
        
        ar_name = self.manifestation_mapper.get_ar_name(manifestation)
        prompt = ""
        
        for example in examples:
            text = example['text'][:80] + "..." if len(example['text']) > 80 else example['text']
            label = example['label']
            
            if label == 1:
                answer = f"نعم، يحتوي على {ar_name}"
            else:
                answer = f"لا، لا يحتوي على {ar_name}"
            
            prompt += f"""النص: "{text}"
السؤال: هل يحتوي على {ar_name}؟
الإجابة: {answer}

"""
        
        return prompt
    
    def parse_response_flexible(self, response: str) -> Tuple[int, str]:
        """Parse model response with balanced precision/recall."""
        response_lower = response.lower().strip()
        reasoning = response
        
        # Check first 100 characters (most important)
        first_part = response_lower[:100]
        
        # Explicit patterns (highest confidence)
        explicit_yes = ['نعم، يحتوي', 'نعم يحتوي', 'يحتوي بالفعل']
        explicit_no = ['لا، لا يحتوي', 'لا يحتوي', 'لا يوجد']
        
        # Check explicit patterns first
        for yes_phrase in explicit_yes:
            if yes_phrase in first_part[:50]:
                return 1, reasoning
        
        for no_phrase in explicit_no:
            if no_phrase in first_part[:50]:
                return 0, reasoning
        
        # Indicators with different weights
        strong_yes = ['نعم', 'يحتوي', 'يوجد']
        strong_no = ['لا،', 'لا.', 'لا يحتوي', 'لا يوجد']
        weak_yes = ['موجود', 'واضح']
        weak_no = ['غير', 'ليس']
        
        # Score calculation (focus on first 50 chars)
        first_50 = first_part[:50]
        yes_score = 0
        no_score = 0
        
        # Strong indicators in first 50 chars (high weight)
        yes_score += sum(4 for word in strong_yes if word in first_50)
        no_score += sum(4 for word in strong_no if word in first_50)
        
        # Weak indicators in first 50 chars (medium weight)
        yes_score += sum(2 for word in weak_yes if word in first_50)
        no_score += sum(2 for word in weak_no if word in first_50)
        
        # Strong indicators in rest of first 100 (lower weight)
        rest_50 = first_part[50:]
        yes_score += sum(1 for word in strong_yes if word in rest_50)
        no_score += sum(1 for word in strong_no if word in rest_50)
        
        # Decision with threshold
        if yes_score > no_score * 1.2:
            return 1, reasoning
        elif no_score > yes_score:
            return 0, reasoning
        else:
            return 0, reasoning  # Default to negative in ties

# Initialize prompter
improved_prompter = ImprovedPrompter(manifestation_mapper)

print("✓ Improved Prompter initialized")

## Part 5: Load Model

Loading AraGPT2-Medium model and tokenizer.

In [ ]:
print("Loading AraGPT2 model and tokenizer...")
print("This may take a few minutes...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16 if config.device == "cuda" else torch.float32,
    device_map="auto" if config.device == "cuda" else None,
    low_cpu_mem_usage=True
)

if config.device == "cpu":
    model = model.to(config.device)

model.eval()

print("=" * 70)
print("✓ Model and tokenizer loaded successfully")
print(f"✓ Model device: {next(model.parameters()).device}")
print(f"✓ Model dtype: {next(model.parameters()).dtype}")
print("=" * 70)

## Part 6: Manifestation Classifier

Main classifier that predicts all 6 manifestations for a given text.

In [ ]:
class ManifestationClassifier:
    """Multi-label classifier for polarization manifestations."""
    
    def __init__(self, model, tokenizer, manifestation_mapper, few_shot_bank, improved_prompter, labels):
        self.model = model
        self.tokenizer = tokenizer
        self.manifestation_mapper = manifestation_mapper
        self.few_shot_bank = few_shot_bank
        self.improved_prompter = improved_prompter
        self.labels = labels
    
    def classify_single_manifestation(self, text: str, manifestation: str, num_few_shot: int = 2) -> Dict:
        """Classify a single manifestation type for the given text."""
        try:
            # Get few-shot examples
            examples = self.few_shot_bank.get_few_shot_examples(manifestation, n=num_few_shot)
            few_shot_text = self.improved_prompter.format_few_shot_with_reasoning(examples, manifestation)
            
            # Create completion prompt
            prompt = self.improved_prompter.create_completion_prompt(text, manifestation, few_shot_text)
            
            # Generate
            inputs = self.tokenizer(prompt, return_tensors="pt", 
                                   max_length=config.max_length, truncation=True).to(config.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs, 
                    max_new_tokens=config.max_new_tokens,
                    temperature=config.temperature,
                    do_sample=True,
                    top_p=config.top_p,
                    pad_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.2
                )
            
            response = self.tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], 
                                            skip_special_tokens=True)
            
            # Parse response
            prediction, reasoning = self.improved_prompter.parse_response_flexible(response)
            
            return {
                'manifestation': manifestation,
                'prediction': prediction,
                'reasoning': response
            }
            
        except Exception as e:
            return {'manifestation': manifestation, 'prediction': 0, 'reasoning': f"Error: {str(e)}"}
    
    def classify_text(self, text: str, num_few_shot: int = 2) -> Dict:
        """Classify all manifestation types for the given text."""
        results = {'text': text, 'predictions': {}, 'manifestation_details': {}}
        
        for manifestation in self.labels:
            manifestation_result = self.classify_single_manifestation(text, manifestation, num_few_shot)
            results['predictions'][manifestation] = manifestation_result['prediction']
            results['manifestation_details'][manifestation] = manifestation_result
        
        return results
    
    def batch_classify(self, texts: List[str], num_few_shot: int = 2, show_progress: bool = True) -> List[Dict]:
        """Classify multiple texts."""
        results = []
        iterator = tqdm(texts, desc="Classifying") if show_progress else texts
        
        for text in iterator:
            try:
                result = self.classify_text(text, num_few_shot)
                results.append(result)
            except Exception as e:
                print(f"Error processing text: {e}")
                results.append({
                    'text': text, 
                    'predictions': {label: 0 for label in self.labels}, 
                    'manifestation_details': {}
                })
        
        return results

# Initialize classifier
classifier = ManifestationClassifier(
    model=model,
    tokenizer=tokenizer,
    manifestation_mapper=manifestation_mapper,
    few_shot_bank=few_shot_bank,
    improved_prompter=improved_prompter,
    labels=MANIFESTATION_TYPES
)

print("✓ Manifestation Classifier initialized and ready")

## Part 7: Test on Sample Texts

Let's test the classifier on a few example texts to see how it works.

In [ ]:
# Test on a few sample texts
test_texts = [
    "احلام انتي ونعالي ومنو انتي حتى تقيمين الفنانين الملكه احلام هههههههه البقره احلام بابا عوفي الفن لااهل الفن",
    "الله يخزي احلام هي والبرنامج الخايس الي كله مصخره",
]

print("=" * 70)
print("TESTING ON SAMPLE TEXTS")
print("=" * 70)

for i, text in enumerate(test_texts, 1):
    print(f"\n{'─' * 70}")
    print(f"Test {i}:")
    print(f"Text: {text}")
    print(f"{'─' * 70}")
    
    result = classifier.classify_text(text, num_few_shot=3)
    
    print("\nPredictions:")
    for manif, pred in result['predictions'].items():
        print(f"  {manif:20s}: {pred}")
    
    print("\nDetailed reasoning:")
    for manif, details in result['manifestation_details'].items():
        if details['prediction'] == 1:
            print(f"\n  ✓ {manif}:")
            print(f"    Response: {details['reasoning'][:100]}...")

print("\n" + "=" * 70)

## Part 8: Prepare Evaluation Dataset

Select a balanced evaluation set from labeled data.

In [ ]:
# Prepare evaluation dataset
print("=" * 70)
print("PREPARING EVALUATION DATASET")
print("=" * 70)

# Get samples with at least one positive label
df_labeled = df[(df[MANIFESTATION_TYPES].notna().all(axis=1)) & 
                (df[MANIFESTATION_TYPES].sum(axis=1) > 0)].copy()

eval_size = min(config.eval_samples, len(df_labeled))
eval_df = df_labeled.sample(n=eval_size, random_state=config.seed)

print(f"\nEvaluation set size: {len(eval_df)} samples")
print("\nLabel distribution in evaluation set:")
for label in MANIFESTATION_TYPES:
    count = eval_df[label].sum()
    print(f"  {label:20s}: {count} ({count/len(eval_df)*100:.1f}%)")

print("\nMulti-label statistics:")
eval_df['label_count'] = eval_df[MANIFESTATION_TYPES].sum(axis=1)
print(f"  Average labels per sample: {eval_df['label_count'].mean():.2f}")
print(f"  Label count distribution:")
print(eval_df['label_count'].value_counts().sort_index().to_string())

print("\n" + "=" * 70)

## Part 9: Run Evaluation

Classify the evaluation set and calculate performance metrics.

In [ ]:
print("=" * 70)
print("RUNNING EVALUATION")
print("=" * 70)
print(f"\nProcessing {len(eval_df)} samples × {len(MANIFESTATION_TYPES)} manifestations")
print(f"Total classifications: {len(eval_df) * len(MANIFESTATION_TYPES)}")
print("\nThis may take several minutes...\n")

# Run batch classification
eval_results = classifier.batch_classify(
    texts=eval_df['text'].tolist(),
    num_few_shot=3,
    show_progress=True
)

print("\n✓ Evaluation complete")
print("=" * 70)

## Part 10: Calculate Metrics

Calculate comprehensive performance metrics.

In [ ]:
# Extract predictions and ground truth
y_true = []
y_pred = []

for idx, row in eval_df.iterrows():
    true_labels = [int(row[label]) for label in MANIFESTATION_TYPES]
    y_true.append(true_labels)

for result in eval_results:
    pred_labels = [result['predictions'][label] for label in MANIFESTATION_TYPES]
    y_pred.append(pred_labels)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate overall metrics
hamming = hamming_loss(y_true, y_pred)
f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0)
f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)
f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0)

print("=" * 70)
print("EVALUATION METRICS")
print("=" * 70)
print("\nOverall Metrics:")
print(f"  Hamming Loss:     {hamming:.4f}")
print(f"  F1 Micro:         {f1_micro:.4f}")
print(f"  F1 Macro:         {f1_macro:.4f}")
print(f"  F1 Samples:       {f1_samples:.4f}")

# Per-manifestation metrics
print("\n" + "─" * 70)
print("Per-Manifestation Metrics:")
print("─" * 70)
print(f"{'Manifestation':<22} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support'}")
print("─" * 70)

per_class_metrics = {}
for i, label in enumerate(MANIFESTATION_TYPES):
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true[:, i], y_pred[:, i], average='binary', zero_division=0
    )
    pos_support = int(y_true[:, i].sum())
    
    # Format label name
    label_name = label.replace('_', ' ').title()
    
    print(f"{label_name:<22} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {pos_support}")
    
    per_class_metrics[label] = {
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'support': pos_support
    }

print("=" * 70)

# Store metrics
metrics_summary = {
    'hamming_loss': float(hamming),
    'f1_micro': float(f1_micro),
    'f1_macro': float(f1_macro),
    'f1_samples': float(f1_samples),
    'per_class': per_class_metrics
}

## Part 11: Visualize Results

Create visualizations of the performance metrics.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Manifestation Classifier Performance Metrics', fontsize=16, fontweight='bold')

# Color palette
colors = ['#e74c3c', '#3498db', '#2ecc71', '#9b59b6', '#f39c12', '#e67e22']

# 1. F1 Scores by Manifestation
ax1 = axes[0, 0]
f1_scores = [per_class_metrics[m]['f1'] for m in MANIFESTATION_TYPES]
labels_formatted = [m.replace('_', '\n').title() for m in MANIFESTATION_TYPES]
bars = ax1.bar(labels_formatted, f1_scores, color=colors, alpha=0.8, edgecolor='black')
ax1.set_title('F1-Score by Manifestation Type', fontsize=13, fontweight='bold')
ax1.set_ylabel('F1-Score', fontsize=11)
ax1.set_ylim(0, 1)
ax1.axhline(y=f1_macro, color='red', linestyle='--', linewidth=2, label=f'F1 Macro: {f1_macro:.3f}')
for bar, score in zip(bars, f1_scores):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{score:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 2. Precision vs Recall
ax2 = axes[0, 1]
precisions = [per_class_metrics[m]['precision'] for m in MANIFESTATION_TYPES]
recalls = [per_class_metrics[m]['recall'] for m in MANIFESTATION_TYPES]
x = np.arange(len(MANIFESTATION_TYPES))
width = 0.35
ax2.bar(x - width/2, precisions, width, label='Precision', color='#3498db', alpha=0.8)
ax2.bar(x + width/2, recalls, width, label='Recall', color='#2ecc71', alpha=0.8)
ax2.set_xticks(x)
ax2.set_xticklabels(labels_formatted, fontsize=9)
ax2.set_title('Precision vs Recall', fontsize=13, fontweight='bold')
ax2.set_ylabel('Score', fontsize=11)
ax2.set_ylim(0, 1)
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. Support (sample count) by manifestation
ax3 = axes[1, 0]
supports = [per_class_metrics[m]['support'] for m in MANIFESTATION_TYPES]
bars = ax3.bar(labels_formatted, supports, color=colors, alpha=0.8, edgecolor='black')
ax3.set_title('Sample Count by Manifestation (Evaluation Set)', fontsize=13, fontweight='bold')
ax3.set_ylabel('Count', fontsize=11)
for bar, count in zip(bars, supports):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.3,
            str(count), ha='center', va='bottom', fontweight='bold', fontsize=10)
ax3.grid(axis='y', alpha=0.3)

# 4. Overall metrics comparison
ax4 = axes[1, 1]
overall_metrics = {
    'F1 Micro': f1_micro,
    'F1 Macro': f1_macro,
    'F1 Samples': f1_samples,
    'Hamming\nLoss': 1 - hamming  # Invert for consistency
}
metric_names = list(overall_metrics.keys())
metric_values = list(overall_metrics.values())
bars = ax4.bar(metric_names, metric_values, color=['#9b59b6', '#e74c3c', '#f39c12', '#2ecc71'], 
               alpha=0.8, edgecolor='black')
ax4.set_title('Overall Performance Metrics', fontsize=13, fontweight='bold')
ax4.set_ylabel('Score', fontsize=11)
ax4.set_ylim(0, 1)
for bar, value in zip(bars, metric_values):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{value:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Visualizations created")

## Part 12: Save Results

Save predictions and metrics to files.

In [ ]:
# Save predictions
results_df = eval_df.copy()

# Add predicted labels
for i, label in enumerate(MANIFESTATION_TYPES):
    results_df[f'{label}_pred'] = y_pred[:, i]

# Add prediction details
for idx, (df_idx, result) in enumerate(zip(eval_df.index, eval_results)):
    for manif in MANIFESTATION_TYPES:
        results_df.loc[df_idx, f'{manif}_reasoning'] = result['manifestation_details'][manif]['reasoning'][:200]

# Save to CSV
predictions_path = 'manifestation_predictions.csv'
results_df.to_csv(predictions_path, index=False, encoding='utf-8')
print(f"✓ Predictions saved to: {predictions_path}")

# Save metrics to JSON
metrics_path = 'manifestation_metrics.json'
with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(metrics_summary, f, indent=2, ensure_ascii=False)
print(f"✓ Metrics saved to: {metrics_path}")

# Create summary report
summary_report = f"""
{'=' * 70}
POLARIZATION MANIFESTATION CLASSIFIER - EVALUATION REPORT
{'=' * 70}

Model: {config.model_name}
Evaluation Samples: {len(eval_df)}
Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

{'─' * 70}
OVERALL METRICS
{'─' * 70}
Hamming Loss:     {hamming:.4f}
F1 Micro:         {f1_micro:.4f}
F1 Macro:         {f1_macro:.4f}
F1 Samples:       {f1_samples:.4f}

{'─' * 70}
PER-MANIFESTATION METRICS
{'─' * 70}
{'Manifestation':<22} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support'}
{'─' * 70}
"""

for label in MANIFESTATION_TYPES:
    metrics = per_class_metrics[label]
    label_name = label.replace('_', ' ').title()
    summary_report += f"{label_name:<22} {metrics['precision']:<12.4f} {metrics['recall']:<12.4f} {metrics['f1']:<12.4f} {metrics['support']}\n"

summary_report += f"""
{'=' * 70}

KEY FINDINGS:
"""

# Add key findings
best_f1 = max(per_class_metrics.items(), key=lambda x: x[1]['f1'])
worst_f1 = min(per_class_metrics.items(), key=lambda x: x[1]['f1'])

summary_report += f"""
- Best performing manifestation: {best_f1[0].replace('_', ' ').title()} (F1: {best_f1[1]['f1']:.4f})
- Most challenging manifestation: {worst_f1[0].replace('_', ' ').title()} (F1: {worst_f1[1]['f1']:.4f})
- Average F1-Score: {f1_macro:.4f}

{'=' * 70}
"""

# Save report
report_path = 'manifestation_evaluation_report.txt'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(summary_report)
print(f"✓ Report saved to: {report_path}")

print("\n" + summary_report)

## Part 13: Interactive Prediction Function

Create a convenient function to classify new texts.

In [ ]:
def predict_manifestations(text: str, show_details: bool = False) -> Dict:
    """
    Predict polarization manifestations for a given Arabic text.
    
    Args:
        text: Arabic text to classify
        show_details: If True, show detailed reasoning for each manifestation
    
    Returns:
        Dictionary with predictions and details
    """
    result = classifier.classify_text(text, num_few_shot=3)
    
    print("=" * 70)
    print("MANIFESTATION PREDICTION")
    print("=" * 70)
    print(f"\nText: {text}\n")
    print("─" * 70)
    print("Predictions:")
    print("─" * 70)
    
    # Count positive predictions
    positive_count = sum(result['predictions'].values())
    
    for manif, pred in result['predictions'].items():
        ar_name = manifestation_mapper.get_ar_name(manif)
        status = "✓" if pred == 1 else "✗"
        label = "YES" if pred == 1 else "NO"
        print(f"  {status} {manif.replace('_', ' ').title():22s} ({ar_name:25s}): {label}")
    
    print("\n" + "─" * 70)
    print(f"Total manifestations detected: {positive_count}/{len(MANIFESTATION_TYPES)}")
    
    if show_details:
        print("\n" + "─" * 70)
        print("Detailed Reasoning:")
        print("─" * 70)
        for manif, details in result['manifestation_details'].items():
            if details['prediction'] == 1:
                ar_name = manifestation_mapper.get_ar_name(manif)
                print(f"\n✓ {manif.replace('_', ' ').upper()} ({ar_name}):")
                print(f"  {details['reasoning'][:150]}...")
    
    print("=" * 70)
    
    return result

print("✓ Interactive prediction function created")
print("\nUsage:")
print("  result = predict_manifestations('your arabic text here')")
print("  result = predict_manifestations('your arabic text here', show_details=True)")

## Part 14: Example Usage

Test the interactive prediction function on custom texts.

In [ ]:
# Example: Test on a custom text
custom_text = "هؤلاء الناس كلهم كسالى ولا يستحقون أي احترام"

result = predict_manifestations(custom_text, show_details=True)

---

## Summary

This notebook implements a multi-label classifier for 6 polarization manifestations in Arabic text:

**Manifestation Types:**
1. **Stereotype (القوالب النمطية)**: Generalizations about groups
2. **Vilification (التشويه والإهانة)**: Abusive or defamatory language
3. **Dehumanization (التجريد من الإنسانية)**: Denying human qualities
4. **Extreme Language (اللغة المتطرفة)**: Inflammatory language
5. **Lack of Empathy (عدم التعاطف)**: Dismissive of others' experiences
6. **Invalidation (الإلغاء والإنكار)**: Denying legitimacy of perspectives

**Key Components:**
- **Model**: AraGPT2-Medium (aubmindlab/aragpt2-medium)
- **Approach**: Few-shot learning with culturally-aware prompting
- **Features**: Balanced response parsing, manifestation-specific hints
- **Evaluation**: Multi-label metrics (Hamming Loss, F1 Micro/Macro/Samples)

**Usage:**
```python
# Classify a single text
result = predict_manifestations("your arabic text here")

# Classify with detailed reasoning
result = predict_manifestations("your arabic text here", show_details=True)

# Batch classify
results = classifier.batch_classify(texts_list, num_few_shot=3)
```

**Output Files:**
- `manifestation_predictions.csv`: Predictions with ground truth
- `manifestation_metrics.json`: Performance metrics in JSON format
- `manifestation_evaluation_report.txt`: Human-readable report